# Import

In [1]:
from numpy.random import seed
seed(888)
from tensorflow import set_random_seed
set_random_seed(404)

C:\Users\b\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\b\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\b\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\b\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
import os
import numpy as np
import tensorflow as tf

from time import strftime

# Constants

In [3]:
X_TRAIN_PATH = 'data/digit_xtrain.csv'
X_TEST_PATH = 'data/digit_xtest.csv'
Y_TRAIN_PATH = 'data/digit_ytrain.csv'
Y_TEST_PATH = 'data/digit_ytest.csv'

LOGGING_PATH = 'tensorboard_mnist_digit_logs/'

NUMBER_CLASSES = 10
VALIDATION_SIZE = 10000

IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_HEIGHT * IMAGE_WIDTH * CHANNELS

# Get the Data

In [4]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)
x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)
x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

Wall time: 26.6 s


# Explore & Visualise Data

รูปภาพเราในที่นี้มีคนจัดการทำให้มันเป็น array มาให้แล้ว ไม่ต้องทำการคอนเวิทเอง มีขนาด 28 * 28 และมี 1 channel
28 * 28 * 1 = 784

In [5]:
print('x_train shape :', x_train_all.shape)
# print(x_train_all[0])
print('x_test shape :', x_test.shape)
# all shape have been flatten, if not >> (number of datapoint, w, h, channel)

x_train shape : (60000, 784)
x_test shape : (10000, 784)


In [6]:
print(y_train_all.shape)

(60000,)


# Data Preprocessing

In [7]:
# Re-scale to 0-1
x_train_all, x_test = x_train_all/255.0, x_test/255.0

In [8]:
values = y_train_all[:5]
np.eye(10)[values]  # given values is consider as position in each row to represent 1 else 0

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

## Conver target values to one-hot encoding

In [9]:
y_train_all = np.eye(NUMBER_CLASSES)[y_train_all]
print(y_train_all.shape)
y_test = np.eye(NUMBER_CLASSES)[y_test]
print(y_test.shape)

print(x_test.shape)

(60000, 10)
(10000, 10)
(10000, 784)


In [10]:
x_validation = x_train_all[:VALIDATION_SIZE]
y_validation = y_train_all[:VALIDATION_SIZE]
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

# Setup Tensorflow Graph

In [11]:
# rerun all below
X = tf.placeholder(tf.float32, shape=[None, TOTAL_INPUTS], name='X')  
# we leave the first dimension black, first dim is how many example we are going to use
# this will actually be determined a little later on, when training model, we will split data to batch
Y = tf.placeholder(tf.float32, shape=[None, NUMBER_CLASSES], name='labels')

## Neural Network Architecture

### Hyperparameters
hyperparameter is something that don't come out of training the model.
A parameter that comes out of training the model would be something like the weights.
So these are things like how long to train our model for, number of epoch used for training, the learning rate use in out optimizer, the number of nodes per layer

all above is hyperparameter

In [12]:
epoch_number = 50
learning_rate = 0.0001  # 1e-4

hidden_1_number = 512
hidden_2_number = 64

we will give some starting values to our weight and bias for neural network.
all the connection weights in the neurals networks need some sort of initail value.
we will give a small random values at the strating point.

In [13]:
initial_w_1 = tf.truncated_normal(shape=[TOTAL_INPUTS, hidden_1_number], stddev=0.1, seed=42)  # the number of input, number of nueron
w1 = tf.Variable(initial_value=initial_w_1)  # create tensorflow's variable to hold all the weight in the first hidden layers
# weight has to be persist and updated as all the calcautions intense floor are going to be run, this is why we carete them as Variabel
initial_bias_1 = tf.constant(value=0.0, shape=[hidden_1_number])# bias is shift of the avtivation function for all node in that layer
b1 = tf.Variable(initial_value=initial_bias_1)
# bias and weight for this first hidden layer will be updated during the training process.
# This is where the learning occur
# These are value, fed into the activation functions of the nuerons and represent the strength of the connection between the units

TensorFlow has this like 2 stage approachs. <br>
The first stage is all set up <br>
The second stage that these calculation are actually all done <br>
So as long as we don't actually tell tensorflow to evaluate any of these tensor and to run all the calculation, we don't actually get to see the value that are generated

In [14]:
initial_w_1

<tf.Tensor 'truncated_normal:0' shape=(784, 512) dtype=float32>

we need to determine how the weight and the bias work together to determine the input in the hidden layer

[w_i] * [o_i] = [in_i]

In [15]:
layer_1_input = tf.matmul(X, w1) + b1  # to multiply 2 matrix 
layer_1_output = tf.nn.relu(layer_1_input)

** challenge ** set up all layer later

In [16]:
initial_w_2 = tf.truncated_normal(shape=[hidden_1_number, hidden_2_number], stddev=0.1, seed=42)
w_2 = tf.Variable(initial_value=initial_w_2)
initial_bias_2 = tf.constant(value=0.0, shape=[hidden_2_number])
b_2 = tf.Variable(initial_value=initial_bias_2)

layer_2_input = tf.matmul(layer_1_output, w_2) + b_2
layer_2_output = tf.nn.relu(layer_2_input)

In [17]:
initial_w_output = tf.truncated_normal(shape=[hidden_2_number, NUMBER_CLASSES], stddev=0.1, seed=42)
w_output = tf.Variable(initial_value=initial_w_output)
initial_bias_output = tf.constant(value=0.0, shape=[NUMBER_CLASSES])
b_output = tf.Variable(initial_value=initial_bias_output)

o_input = tf.matmul(layer_2_output, w_output) + b_output
output = tf.nn.softmax(o_input)

# Tensorboard Setup

In [18]:
name = 'model'
folder_name = f'Model {name} at {strftime("%D %H%M")}'.replace('/', ';')
directory = os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory)
except OSError as oserr:
    print('error :', oserr.strerror)
else:
    print('Successfully create file')

Successfully create file


# Loss, Optimisation and Metric

In [19]:
# create loss function >> google >> tensorflow loss funtion
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output))
# this loss calculation work for calculating the loss on the entire dataset
# it will not give a good result when we've got individual batches
# so when we have individual batches is we need to take the average of the losses
# by using tf.reduce_mean

In [20]:
# create Optimisation >> google >> tensorflow Optimisation
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_step = optimizer.minimize(loss)
# all above is very quite complex, this is why we have keras lol

## Accuracy Metric

In [21]:
# to create accuracy we need to compare 2 things, the predictions and true labels
correct_pred = tf.equal(tf.argmax(output, axis=1), tf.argmax(Y, axis=1))
# get the maximun from both prediction values and labels

accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [22]:
tf.summary.scalar('accuracy', accuracy)
tf.summary.scalar('loss', loss)

<tf.Tensor 'loss:0' shape=() dtype=string>

# Run session

Tensorflow has session object. <br>
Session objects encapsulate the environment under which all the operation and all the calculations take place and are executed. <br>
All of setting, calculation take place inisde a session.

In [23]:
session = tf.Session()
# initialise all the variables
init = tf.global_variables_initializer()
session.run(init)

### Setup Filewriter and Merge Summaries

In [24]:
merged_summary = tf.summary.merge_all()  # all the calcalation whta we want to read, we will combine and merge them and store it to this var
train_writer = tf.summary.FileWriter(directory + '/train')
train_writer.add_graph(session.graph)
# all of the calculation are in thing called 'graph' >> graph
# a calculation are evaluate during the session >> session.graph 
# for a file writer to know which calculation to save and write down to the disk
# we have to tell it which graph to use [].add_graph

In [25]:
validation_writer = tf.summary.FileWriter(directory + '/validation')
# in the theory i should add_graph to the validation_writer as well
# but validation_writer and train_writer actually using the smae graph
# validation_writer.add_graph(session.grapg)

#### to See the values inside the var that we created

In [26]:
# now if you want to see the value inside tensor. >> evaluate >> eval(session)
b_2.eval(session)
# it's like a pipe when we actually run the session the data start flowing through these pipe
# then we can evaluate the calculation and get output

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

## Batching the Data

In [27]:
size_of_batch = 1000
examples_number = y_train.shape[0]
iteration_number = int(examples_number/size_of_batch)

epoch_index = 0

In [28]:
def get_batch(size_of_batch, data, labels):
    global examples_number
    global epoch_index
    
    start = epoch_index
    epoch_index += size_of_batch
    if epoch_index > examples_number:
        start = 0
        epoch_index = size_of_batch
    end = epoch_index
    
    return data[start:end], labels[start:end]  # 0:1000(999), 1000:2000(1999), ...

# Training Loop

In [29]:
for epoch in range(epoch_number):
    for i in range(iteration_number):
        batch_x, batch_y = get_batch(size_of_batch, x_train, y_train)
        feed_dictionary = {X:batch_x, Y:batch_y}
        # running optimizer, running calcalation on given data
        session.run(train_step, feed_dict=feed_dictionary)
        
        # using run method can give us an accuracy as output
        # specify what the session should fetch, what output that we want to get >> fetchs
        # fetch is when we asking our session to return some calculations
        # accuracy is going to be calcalated on out batch
    #batch_accuracy = session.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictionary)  # like keras callbacks
    s, batch_accuracy = session.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictionary)
    train_writer.add_summary(s, epoch)
    print(f'Epoch : {epoch} \t| Training Accuracy : {batch_accuracy.round(3)}')
    
    # validation
    summary = session.run(fetches=merged_summary, feed_dict={X:x_validation, Y:y_validation})
    validation_writer.add_summary(summary, epoch)
    
print('Done training')

Epoch : 0 	| Training Accuracy : 0.6380000114440918
Epoch : 1 	| Training Accuracy : 0.8040000200271606
Epoch : 2 	| Training Accuracy : 0.8450000286102295
Epoch : 3 	| Training Accuracy : 0.8600000143051147
Epoch : 4 	| Training Accuracy : 0.871999979019165
Epoch : 5 	| Training Accuracy : 0.8759999871253967
Epoch : 6 	| Training Accuracy : 0.878000020980835
Epoch : 7 	| Training Accuracy : 0.8809999823570251
Epoch : 8 	| Training Accuracy : 0.8820000290870667
Epoch : 9 	| Training Accuracy : 0.8840000033378601
Epoch : 10 	| Training Accuracy : 0.8849999904632568
Epoch : 11 	| Training Accuracy : 0.906000018119812
Epoch : 12 	| Training Accuracy : 0.9570000171661377
Epoch : 13 	| Training Accuracy : 0.9639999866485596
Epoch : 14 	| Training Accuracy : 0.9660000205039978
Epoch : 15 	| Training Accuracy : 0.968999981880188
Epoch : 16 	| Training Accuracy : 0.972000002861023
Epoch : 17 	| Training Accuracy : 0.972000002861023
Epoch : 18 	| Training Accuracy : 0.972000002861023
Epoch : 19

## Reset for the Next Run

In [31]:
train_writer.close()
validation_writer.close()
session.close()
tf.reset_default_graph()